<a href="https://colab.research.google.com/github/rasaghnak/leetcode_practise/blob/main/Radius_geopython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas geopandas folium nltk ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium import IFrame
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import random

# Generate random geospatial data for the example
data = {
    "Organization Name": [f"Utility_{i}" for i in range(1, 101)],
    "Contact Person": [f"Contact_{i}" for i in range(1, 101)],
    "Title": ["Manager", "Director", "Supervisor", "Engineer", "Technician"] * 20,
    "Email": [f"contact{i}@utility.com" for i in range(1, 101)],
    "Phone Number": [f"{random.randint(100, 999)}-555-{random.randint(1000, 9999)}" for i in range(1, 101)],
    "Address": [f"{random.randint(1, 999)} Main St, Town_{i}, USA" for i in range(1, 101)],
    "Website": [f"www.utility{i}.com" for i in range(1, 101)],
    "comments": [f"Issue_{i}: {random.choice(['High lead levels', 'Pipeline leakage', 'Water contamination', 'Old infrastructure', 'Insufficient funding'])}" for i in range(1, 101)],
    "Latitude": [random.uniform(38.0, 40.0) for _ in range(100)],
    "Longitude": [random.uniform(-87.0, -85.0) for _ in range(100)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Display the first few rows of the GeoDataFrame
gdf.head()


,Organization Name,Contact Person,Title,Email,Phone Number,Address,Website,comments,Latitude,Longitude,geometry
0,Utility_1,Contact_1,Manager,contact1@utility.com,485-555-9131,"962 Main St, Town_1, USA",www.utility1.com,Issue_1: Pipeline leakage,39.341475,-85.766829,POINT (-85.76683 39.34147)
1,Utility_2,Contact_2,Director,contact2@utility.com,596-555-1635,"460 Main St, Town_2, USA",www.utility2.com,Issue_2: Old infrastructure,38.313898,-86.604614,POINT (-86.60461 38.31390)
2,Utility_3,Contact_3,Supervisor,contact3@utility.com,965-555-7344,"276 Main St, Town_3, USA",www.utility3.com,Issue_3: Old infrastructure,38.395118,-85.718030,POINT (-85.71803 38.39512)
3,Utility_4,Contact_4,Engineer,contact4@utility.com,293-555-9078,"990 Main St, Town_4, USA",www.utility4.com,Issue_4: Old infrastructure,38.142069,-86.802975,POINT (-86.80297 38.14207)
4,Utility_5,Contact_5,Technician,contact5@utility.com,821-555-4891,"48 Main St, Town_5, USA",www.utility5.com,Issue_5: Pipeline leakage,39.855653,-86.467314,POINT (-86.46731 39.85565)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and word not in string.punctuation]
    return tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def find_utility_by_location(query, gdf):
    tokens = preprocess(query)
    relevant_data = gdf[gdf['comments'].str.contains('|'.join(tokens), case=False, na=False)]
    return relevant_data

def create_map(data):
    if data.empty:
        return "No relevant utilities found for the given query."

    center_lat = data['Latitude'].mean()
    center_lon = data['Longitude'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

    for _, row in data.iterrows():
        popup_content = f"Organization: {row['Organization Name']}<br>Comment: {row['comments']}"
        popup = folium.Popup(IFrame(popup_content, width=300, height=100), max_width=300)
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup).add_to(m)

    return m

def chatbot_response(query):
    relevant_data = find_utility_by_location(query, gdf)
    return create_map(relevant_data)

# Test the chatbot
query = "Show me issues with water contamination"
map_result = chatbot_response(query)
map_result


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='Type your query here...',
    description='Query:',
    disabled=False
)

# Create an output widget to display the map
output = widgets.Output()

def on_submit(change):
    with output:
        clear_output()
        query = text_input.value
        response = chatbot_response(query)
        if isinstance(response, str):
            print(response)
        else:
            display(response)

text_input.on_submit(on_submit)

# Display the widgets
display(text_input, output)


Text(value='', description='Query:', placeholder='Type your query here...')

Output()